In [66]:
import numpy as np
import pandas as pd
from datetime import datetime, UTC

METRIC_REGISTRY = {
    "electricity_usage": {
        "metrics": [
            {"metric_key": "electricity_total_kwh", "op": "sum",  "field": "Usage_kWh", "unit": "kwh"},
            {"metric_key": "electricity_avg_hourly_kwh",    "op": "mean", "field": "Usage_kWh", "unit": "kwh_per_hour"},
            {"metric_key": "electricity_peak_hourly_kwh",   "op": "max",  "field": "Usage_kWh", "unit": "kwh_per_hour"},
        ],
    },
    "citygas_usage": {
        "constants": {"gj_per_m3": 0.043, "tco2e_per_m3": 0.00245},
        "metrics": [
            {"metric_key": "citygas_total_m3", "op": "sum", "field": "flow_m3", "unit": "m3"},
            {"metric_key": "citygas_avg_hourly_m3",    "op": "mean","field": "flow_m3", "unit": "m3_per_hour"},
            {"metric_key": "citygas_peak_hourly_m3",   "op": "max", "field": "flow_m3", "unit": "m3_per_hour"},
            {"metric_key": "citygas_total_energy_gj",  "op": "sum_mul_const", "field": "flow_m3", "const": "gj_per_m3", "unit": "gj"},
            {"metric_key": "scope1_total_tco2e",       "op": "sum_mul_const", "field": "flow_m3", "const": "tco2e_per_m3", "unit": "tco2e"},
        ],
    },
    "water_usage": {
        "metrics": [
            {"metric_key": "water_total_m3", "op": "sum",  "field": "flow_m3", "unit": "m3"},
            {"metric_key": "water_avg_hourly_m3",    "op": "mean", "field": "flow_m3", "unit": "m3_per_hour"},
            {"metric_key": "water_peak_hourly_m3",   "op": "max",  "field": "flow_m3", "unit": "m3_per_hour"},
            {"metric_key": "water_usage_cv",         "op": "cv",   "field": "flow_m3", "unit": "ratio"},
        ],
    },
}

OPS = {
    "sum": lambda s: float(np.nansum(s)),
    "mean": lambda s: float(np.nanmean(s)),
    "max": lambda s: float(np.nanmax(s)),
    "cv": lambda s: float(np.nanstd(s) / np.nanmean(s)) if np.nanmean(s) else None,
    "sum_mul_const": lambda s, c: float(np.nansum(s) * c),
}

def _parse_granularity_to_hours(g: str) -> float:
    g = (g or "").strip().lower()
    mapping = {
        "10min": 10 / 60.0,
        "15min": 15 / 60.0,
        "30min": 30 / 60.0,
        "hourly": 1.0,
        "day": 24.0,
        "week": 24.0 * 7.0,
        "month": 24.0 * 30.0,
    }
    return mapping.get(g, 1.0)

def _to_multiplier(unit: str, target_unit: str) -> float:
    u = (unit or "").strip().lower()
    t = (target_unit or "").strip().lower()

    if not u or u in ("-", "time") or u == t:
        return 1.0

    if u == "wh" and t == "kwh":
        return 1 / 1000.0
    if u == "mwh" and t == "kwh":
        return 1000.0
    if u == "kwh" and t == "wh":
        return 1000.0

    if u == "l" and t == "m3":
        return 1 / 1000.0
    if u == "m3" and t == "l":
        return 1000.0

    return 1.0

def _unit_map_from_input2(input2: dict) -> dict:
    fields = input2["payload"].get("validated_fields", [])
    schema = input2["payload"].get("unit_schema", [])
    return {f: schema[i] if i < len(schema) else "-" for i, f in enumerate(fields)}

def _apply_base_unit_conversion(df: pd.DataFrame, unit_map: dict, base_unit_by_field: dict) -> pd.DataFrame:
    df = df.copy()
    for field, target_unit in base_unit_by_field.items():
        if field in df.columns:
            mult = _to_multiplier(unit_map.get(field, "-"), target_unit)
            df[field] = pd.to_numeric(df[field], errors="coerce") * mult
    return df

def build_output(input1: dict, input2: dict, metrics: list[dict]) -> dict:
    slot = input1["slotName"]
    return {
        "status": "OK",
        "period": {"start": input1.get("period_start"), "end": input1.get("period_end")},
        "source": {"slotName": slot, "file_path": input2["file_path"]},
        "metrics": metrics,
        "processed_at": input2.get("processed_at") or datetime.now(UTC).isoformat(),
    }

def make_metrics(input1: dict, input2: dict):
    slot = input1["slotName"]
    df = input1["dataframe"].copy()
    cfg = METRIC_REGISTRY[slot]

    # 1) validated_fields로 컬럼명 교체
    validated = input2["payload"].get("validated_fields", [])
    if len(validated) != len(df.columns):
        raise ValueError("validated_fields length mismatch with df columns")
    df.columns = validated

    # 2) timestamp → datetime
    if "timestamp" not in df.columns:
        raise ValueError("timestamp column missing")
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["timestamp"])

    # 3) period 필터
    start = pd.to_datetime(input1.get("period_start"), errors="coerce")
    end = pd.to_datetime(input1.get("period_end"), errors="coerce")
    if pd.notna(start):
        df = df[df["timestamp"] >= start]
    if pd.notna(end):
        df = df[df["timestamp"] < end]
    if df.empty:
        return []

    # 4) unit_schema 기반 base unit 변환
    unit_map = _unit_map_from_input2(input2)

    base_unit_by_field = {}
    for m in cfg["metrics"]:
        if m["unit"] in ("kwh", "m3", "gj", "tco2e"):
            base_unit_by_field[m["field"]] = m["unit"]
        elif m["unit"] in ("kwh_per_hour", "m3_per_hour"):
            base_unit_by_field[m["field"]] = m["unit"].split("_per_hour")[0]

    df = _apply_base_unit_conversion(df, unit_map, base_unit_by_field)

    # 5) 시간당 환산을 위한 granularity
    interval_hours = _parse_granularity_to_hours(input2["payload"].get("time_granularity", "hourly"))
    to_per_hour = (1.0 / interval_hours) if interval_hours else 1.0

    out = []
    for m in cfg["metrics"]:
        field = m["field"]
        if field not in df.columns:
            continue

        s = df[field].to_numpy(dtype=float, copy=False)

        # 6) 기본 집계
        if m["op"] == "sum_mul_const":
            c = cfg["constants"][m["const"]]
            val = OPS[m["op"]](s, c)
        else:
            val = OPS[m["op"]](s)

        # 7) per_hour 처리 
        if m["unit"].endswith("_per_hour") and val is not None:
            if m["op"] in ("sum", "sum_mul_const"):
                val = val * to_per_hour

            elif m["op"] == "max":
                # 시간별(sum)로 만든 뒤 peak
                hourly = df.set_index("timestamp")[field].resample("H").sum()
                val = float(np.nanmax(hourly.to_numpy())) if len(hourly) else None

            elif m["op"] == "mean":
                # 시간별(sum)로 만든 뒤 평균
                hourly = df.set_index("timestamp")[field].resample("H").sum()
                val = float(np.nanmean(hourly.to_numpy())) if len(hourly) else None

        gran = input2["payload"].get("time_granularity")

        metric_key = m["metric_key"]
        if m.get("unit") == "ratio" and gran:
            metric_key = f"{metric_key} ({gran})"

        out.append({
            "metric_key": metric_key,
            "value": val,
            "unit": m["unit"],
        })

    return build_output(input1, input2, out)


In [67]:
## 테스트용 임시 데이터
import pandas as pd
import numpy as np
from datetime import datetime

def run_metric_test(slot_name: str, path: str | None = None):
    # 1) df 준비: path 있으면 xlsx, 없으면 테스트 데이터
    df = pd.read_excel(path)
    file_path = path.split("/")[-1]

    # 2) input1/input2 구성
    input1 = {
        "slotName": slot_name,
        "kind": "EXCEL",
        "ext": "xlsx",
        "period_start": "2025-10-01T00:00:00",
        "period_end": "2025-12-31T23:00:00",
        "dataframe": df,
    }
    input2 = {
        "status": "PASS",
        "file_path": file_path,
        "payload": {"time_granularity": "15min",
                    "unit_schema": ["time","m3","-","-","-",],
                    "validated_fields": ["timestamp", *df.columns[1:]]},
        "processed_at": datetime.utcnow().isoformat() + "Z",
    }


    # 3) metrics + output 생성
    metrics = make_metrics(input1, input2)
    return input1, input2


In [68]:
# 사용
input1, input2 = run_metric_test("citygas_usage", "test_file/xlsx_gas.xlsx")
out = make_metrics(input1, input2)
display(out)


C:\Users\User\AppData\Local\Temp\ipykernel_59836\285173634.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat() + "Z",
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:163: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample(

{'status': 'OK',
 'period': {'start': '2025-10-01T00:00:00', 'end': '2025-12-31T23:00:00'},
 'source': {'slotName': 'citygas_usage', 'file_path': 'xlsx_gas.xlsx'},
 'metrics': [{'metric_key': 'citygas_total_m3',
   'value': 18480.000000000007,
   'unit': 'm3'},
  {'metric_key': 'citygas_avg_hourly_m3',
   'value': 25.66666666666668,
   'unit': 'm3_per_hour'},
  {'metric_key': 'citygas_peak_hourly_m3',
   'value': 38.74007816493066,
   'unit': 'm3_per_hour'},
  {'metric_key': 'citygas_total_energy_gj',
   'value': 794.6400000000002,
   'unit': 'gj'},
  {'metric_key': 'scope1_total_tco2e',
   'value': 45.27600000000002,
   'unit': 'tco2e'}],
 'processed_at': '2026-01-20T08:20:25.771259Z'}

In [69]:
# 사용
input1, input2 = run_metric_test("electricity_usage", "test_file/xlsx_elec.xlsx")
out = make_metrics(input1, input2)
display(out)

C:\Users\User\AppData\Local\Temp\ipykernel_59836\285173634.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat() + "Z",
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:163: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample(

{'status': 'OK',
 'period': {'start': '2025-10-01T00:00:00', 'end': '2025-12-31T23:00:00'},
 'source': {'slotName': 'electricity_usage', 'file_path': 'xlsx_elec.xlsx'},
 'metrics': [{'metric_key': 'electricity_total_kwh',
   'value': 134339.9999999997,
   'unit': 'kwh'},
  {'metric_key': 'electricity_avg_hourly_kwh',
   'value': 186.58333333333292,
   'unit': 'kwh_per_hour'},
  {'metric_key': 'electricity_peak_hourly_kwh',
   'value': 303.710042310757,
   'unit': 'kwh_per_hour'}],
 'processed_at': '2026-01-20T08:20:27.035171Z'}

In [70]:
# 사용
input1, input2 = run_metric_test("water_usage", "test_file/xlsx_water.xlsx")
out = make_metrics(input1, input2)
display(out)


C:\Users\User\AppData\Local\Temp\ipykernel_59836\285173634.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat() + "Z",
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:163: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample("H").sum()
C:\Users\User\AppData\Local\Temp\ipykernel_59836\431754625.py:168: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df.set_index("timestamp")[field].resample(

{'status': 'OK',
 'period': {'start': '2025-10-01T00:00:00', 'end': '2025-12-31T23:00:00'},
 'source': {'slotName': 'water_usage', 'file_path': 'xlsx_water.xlsx'},
 'metrics': [{'metric_key': 'water_total_m3',
   'value': 2770.000000000001,
   'unit': 'm3'},
  {'metric_key': 'water_avg_hourly_m3',
   'value': 3.8472222222222237,
   'unit': 'm3_per_hour'},
  {'metric_key': 'water_peak_hourly_m3',
   'value': 6.640630453014286,
   'unit': 'm3_per_hour'},
  {'metric_key': 'water_usage_cv (15min)',
   'value': 0.47103278418441624,
   'unit': 'ratio'}],
 'processed_at': '2026-01-20T08:20:46.172524Z'}